In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizerFast, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import numpy as np
from typing import List, Tuple

In [ ]:
# Definição de caminhos dos arquivos de corpus
train_file = "Penn Treebank/Secs0-18 - training"
dev_file   = "Penn Treebank/Secs19-21 - development"
test_file  = "Penn Treebank/Secs22-24 - testing"

In [ ]:
# ------------------------------
# Funções de pré-processamento do texto
def carregar_corpus(caminho_arquivo: str) -> str:
    """
    Lê o arquivo completo em utf-8 e retorna como string.
    """
    with open(caminho_arquivo, "r", encoding="utf-8") as f:
        return f.read()

def dividir_em_sentencas(texto: str) -> List[str]:
    """
    Divide o texto em sentenças, assumindo uma sentença por linha.
    """
    return texto.strip().split("\n")

def processar_sentenca(sentenca: str) -> List[Tuple[str, str]]:
    """
    Separa tokens de formato palavra_TAG em pares (palavra, tag).
    Converte para lowercase, exceto nomes próprios (NNP, NNPS).
    """
    tokens = sentenca.strip().split()
    pares = []
    for token in tokens:
        if "_" in token:
            palavra, tag = token.rsplit("_", 1)
            if not(tag == 'NNP' or tag == 'NNPS'):
                palavra = palavra.lower()
            pares.append((palavra, tag))
    return pares

def construir_dataframe(sentencas: List[str]) -> pd.DataFrame:
    """
    Cria um DataFrame 'longo' com colunas:
    sentenca (ID), palavra, tag e posicao_na_sentenca.
    """
    dados = []
    for sent_id, sentenca in enumerate(sentencas):
        palavras_tags = processar_sentenca(sentenca)
        for posicao, (palavra, tag) in enumerate(palavras_tags):
            dados.append({
                "sentenca": sent_id + 1,
                "palavra": palavra,
                "tag": tag,
                "posicao_na_sentenca": posicao
            })

    return pd.DataFrame(dados)

In [ ]:
# Carregando e processando os datasets
texto_raw_train = carregar_corpus(train_file)
sentencas_train = dividir_em_sentencas(texto_raw_train)
df_treino = construir_dataframe(sentencas_train)

# Dev + Test combinados para avaliação final
texto_raw_dev = carregar_corpus(dev_file)
texto_raw_teste = carregar_corpus(test_file)
sentencas_teste = dividir_em_sentencas(texto_raw_dev + texto_raw_teste)
df_teste = construir_dataframe(sentencas_teste)

# Exibe as primeiras linhas para verificação rápida
df_treino

,sentenca,palavra,tag,posicao_na_sentenca
0,1,Pierre,NNP,0
1,1,Vinken,NNP,1
2,1,",",",",2
3,1,61,CD,3
4,1,years,NNS,4
...,...,...,...,...
912339,38219,to,TO,21
912340,38219,San,NNP,22
912341,38219,Francisco,NNP,23
912342,38219,instead,RB,24


In [ ]:
# Agrupar dados por sentença
df_treino_agrupado = df_treino.groupby('sentenca').agg({'palavra': list, 'tag': list}).reset_index()
df_teste_agrupado = df_teste.groupby('sentenca').agg({'palavra': list, 'tag': list}).reset_index()

encoder = LabelEncoder()
encoder.fit(df_treino["tag"])

def encode_tags(tags):
    return encoder.transform(tags)

df_treino_agrupado['tags'] = df_treino_agrupado['tag'].apply(encode_tags)
df_teste_agrupado['tags'] = df_teste_agrupado['tag'].apply(encode_tags)

# Criar datasets
dataset_train = Dataset.from_pandas(df_treino_agrupado[['palavra', 'tags']])
dataset_test = Dataset.from_pandas(df_teste_agrupado[['palavra', 'tags']])


In [ ]:
# 2. Tokenizador com tratamento especial para sequence labeling
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["palavra"],
        truncation=True,
        padding="max_length",
        max_length=128,  # Reduzido para economia de memória
        is_split_into_words=True  # Indica que já temos tokens separados
    )

    labels = []
    for i, tags in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Tokens especiais (CLS, SEP, PAD) recebem -100 (ignorar na loss)
            if word_idx is None:
                label_ids.append(-100)
            # Nova palavra: usar o label correto
            elif word_idx != previous_word_idx:
                label_ids.append(tags[word_idx])
            # Mesma palavra: manter ou ignorar (aqui ignoramos subtokens)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_test = dataset_test.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/38219 [00:00<?, ? examples/s]

Map:   0%|          | 0/10989 [00:00<?, ? examples/s]

In [ ]:
# 3. CORREÇÃO: Modelo para sequence labeling
class BERTForPOSTagging(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))

        return {"loss": loss, "logits": logits}

# Usar número correto de classes
num_labels = len(encoder.classes_)
model = BERTForPOSTagging(num_labels=num_labels)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# 4. CORREÇÃO: Função de métricas para sequence labeling
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remover tokens ignorados (onde label = -100)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Achatar listas
    true_predictions = sum(true_predictions, [])
    true_labels = sum(true_labels, [])

    # Calcular acurácia
    accuracy = (np.array(true_predictions) == np.array(true_labels)).mean()
    return {"accuracy": accuracy}

# Configurar treinamento
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Reduzido para caber na memória
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

# Treinar modelo
trainer.train()

Step,Training Loss
100,1.698400
200,0.310000
300,0.202000
400,0.166000
500,0.136400
600,0.124000
700,0.116700
800,0.113900
900,0.115000
1000,0.108600


TrainOutput(global_step=7167, training_loss=0.1005713277004812, metrics={'train_runtime': 2565.5944, 'train_samples_per_second': 44.69, 'train_steps_per_second': 2.794, 'total_flos': 0.0, 'train_loss': 0.1005713277004812, 'epoch': 3.0})

In [ ]:
eval_result = trainer.evaluate()
print("Acurácia:", eval_result)

Acurácia: {'eval_loss': 0.07968544214963913, 'eval_accuracy': 0.9745890130574674, 'eval_runtime': 94.3861, 'eval_samples_per_second': 116.426, 'eval_steps_per_second': 7.279, 'epoch': 3.0}


In [ ]:
# Função para fazer previsões em novas sentenças
def predict_pos(sentence: str) -> List[Tuple[str, str]]:
    # Pré-processamento da sentença
    tokens = [token.lower() for token in sentence.split()]

    # Tokenização com BERT
    inputs = tokenizer(
        tokens,
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
        return_tensors="pt"
    )

    # Fazer previsão
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    # Obter as previsões
    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    # Alinhar tokens com previsões
    word_ids = inputs.word_ids(batch_index=0)
    previous_word_idx = None
    tagged_sentence = []

    for i, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue  # Ignorar tokens especiais e subtokens

        word = tokens[word_idx]
        tag_id = predictions[i]
        tag = encoder.inverse_transform([tag_id])[0]

        tagged_sentence.append((word, tag))
        previous_word_idx = word_idx

    return tagged_sentence

# 1. Avaliar o modelo no conjunto de teste
results = trainer.evaluate(tokenized_test)
print("\nResultados da avaliação:")
print(f"Acurácia: {results['eval_accuracy']:.4f}")
print(f"Perda: {results['eval_loss']:.4f}")

# 2. Testar com exemplos individuais
test_sentences = [
    "The quick brown fox jumps over the lazy dog",
    "Apple is looking at buying U.K. startup for $1 billion",
    "I have lived in New York for 5 years"
]

print("\nTestes com sentenças de exemplo:")
for sentence in test_sentences:
    prediction = predict_pos(sentence)
    print(f"\nSentença: {sentence}")
    print("Previsões:")
    for word, tag in prediction:
        print(f"{word}: {tag}")

# 3. Avaliação detalhada com relatório de classificação
from sklearn.metrics import classification_report

def get_true_pred_labels(dataset):
    true_labels = []
    pred_labels = []

    for example in dataset:
        inputs = {
            "input_ids": torch.tensor([example["input_ids"]]),
            "attention_mask": torch.tensor([example["attention_mask"]])
        }

        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)

        predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

        # Filtrar tokens relevantes
        for i, label_id in enumerate(example["labels"]):
            if label_id != -100:  # Ignorar tokens especiais
                true_labels.append(label_id)
                pred_labels.append(predictions[i])

    return true_labels, pred_labels

# Obter labels verdadeiras e previstas
true_labels, pred_labels = get_true_pred_labels(tokenized_test)

# Decodificar labels
true_tags = encoder.inverse_transform(true_labels)
pred_tags = encoder.inverse_transform(pred_labels)

# Gerar relatório
print("\nRelatório de Classificação Detalhado:")
print(classification_report(true_tags, pred_tags, zero_division=0))

# 4. Matriz de confusão (opcional)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Filtrar para as 20 tags mais comuns (para visualização)
common_tags = np.array(encoder.classes_)[np.argsort(np.bincount(true_labels))[-20:]]
mask = np.isin(true_tags, common_tags)

cm = confusion_matrix(true_tags[mask], pred_tags[mask], labels=common_tags)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=common_tags, yticklabels=common_tags)
plt.title('Matriz de Confusão (20 tags mais comuns)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

KeyboardInterrupt: 